<a href="https://colab.research.google.com/github/NamanNYC/Final_Project/blob/database2/scrape%26dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initiate Connection to PostgreSQL and RDS**

In [1]:

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-19 06:13:18--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.70MB/s    in 0.6s    

2021-09-19 06:13:20 (1.70 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
connection_string = 'lmpt-finalproject.coke2w4vs8wf.us-east-2.rds.amazonaws.com'
password = #withheld
database_name = 'postgres'

# Configure settings for RDS
mode = "append"
jdbc_url=f"jdbc:postgresql://{connection_string}:5432/{database_name}"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

**Import Links to Data**

In [4]:
import pandas as pd
table_dfs = pd.read_html('https://wildfire.alberta.ca/resources/historical-data/historical-wildfire-database.aspx')
table_df = table_dfs[0]

In [5]:
table_df

,Date Range,Data Download,Data Dictionary
0,2006 - 2018,fires_2006to2018.csv,data_dictionary_2006to2018.pdf
1,1996 – 2005,fires_1996to2005.csv,data_dictionary_1996to2005.pdf
2,1983 – 1995,fires_1983to1995.csv,data_dictionary_1983to1995.pdf
3,1961 – 1982,fires_1961to1982.csv,data_dictionary_1961to1982.pdf


In [6]:
table_df['BaseLink'] = [f'https://wildfire.alberta.ca/resources/historical-data/documents/{link}' for link in table_df['Data Download']]

In [7]:
table_df

,Date Range,Data Download,Data Dictionary,BaseLink
0,2006 - 2018,fires_2006to2018.csv,data_dictionary_2006to2018.pdf,https://wildfire.alberta.ca/resources/historic...
1,1996 – 2005,fires_1996to2005.csv,data_dictionary_1996to2005.pdf,https://wildfire.alberta.ca/resources/historic...
2,1983 – 1995,fires_1983to1995.csv,data_dictionary_1983to1995.pdf,https://wildfire.alberta.ca/resources/historic...
3,1961 – 1982,fires_1961to1982.csv,data_dictionary_1961to1982.pdf,https://wildfire.alberta.ca/resources/historic...


In [14]:
# Import CSVs into Datatbase via a For Loop
from pyspark import SparkFiles
from pyspark.sql import SparkSession

for url, name in zip(table_df['BaseLink'],table_df['Data Download']):
  
    spark = SparkSession.builder.appName("LMPT-Forest-Fires").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
    tablename = name.replace(".csv","")
    spark.sparkContext.addFile(url)
    df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
    df.write.jdbc(url=jdbc_url, table=tablename, mode=mode, properties=config)